In [1]:
# 한국어 형태소 분석기
## kiwipiepy.Kiwi() 사용
## .tokenize() 함수를 이용해서 분석
## 분석 결과는 해당 형태소(.form), 형태소 품사(.tag), 형태소 위치(.start), 형태소 길이(.len) 를 같이 출력
## 형태소만 따로 골라내려면 for문을 이용해서 별개로 추출해야 함

import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
import numpy
import pandas
from IPython.display import display

# from konlpy.tag import Kkma
from kiwipiepy import Kiwi #클래스 형태로 구현됨
from tqdm import tqdm
# import threading

In [2]:
# 사용 방법
kiwi = Kiwi() # 클래스를 객체로 생성하는데 시간이 꽤 걸려서 객체를 생성하고 시작해야함

text = "이 영화는 재미있습니다. 추천합니다."
results = kiwi.tokenize(text)
print(results)

#단어만 추출
words = [] #단어 저장 리스트 따로 선언
for i in range(len(results)) :
    words.append(results[i].form) #for 문 내에서 하나씩 집어넣어야 함

print(words)
print(" ".join(words))

[Token(form='이', tag='MM', start=0, len=1), Token(form='영화', tag='NNG', start=2, len=2), Token(form='는', tag='JX', start=4, len=1), Token(form='재미있', tag='VA', start=6, len=3), Token(form='습니다', tag='EF', start=9, len=3), Token(form='.', tag='SF', start=12, len=1), Token(form='추천', tag='NNG', start=14, len=2), Token(form='하', tag='XSV', start=16, len=1), Token(form='ᆸ니다', tag='EF', start=16, len=3), Token(form='.', tag='SF', start=19, len=1)]
['이', '영화', '는', '재미있', '습니다', '.', '추천', '하', 'ᆸ니다', '.']
이 영화 는 재미있 습니다 . 추천 하 ᆸ니다 .


In [3]:
# 구동하다가 분석하는데 10초 이상 걸리는 문장을 넘기는 서브스레드를 가진 분석 class를 만들어 보도록 합시다
# kiwi는 속도가 빨라서 필요가 없습니다.

# class Preprocessor() :
#     def __init__(self) :
#         self._result = None #스레드 결과값 저장 변수
#         self._error = None #스레드 오류 발생시 오류 저장 변수
#     def _process_with_timeout(self, s : str, timeout : int) :
#         def work() :
#             try :
#                 self._result = Kkma().morphs(s)
#             except Exception as e:
#                 self._error = e
#         self._error = None
#         thread = threading.Thread(target = work, daemon = True) #work 함수 구동할 스레드 선언
#         thread.start() #해당 스레드 동작 시작
#         thread.join(timeout = timeout) #해당 스레드 timeout초동안 기다려 주고

#         if thread.is_alive() : #기다렸는데도 아직 동작이 안끝났으면
#             raise Exception(f"Error : {timeout}초 시간 초과")
#         if self._error is not None :
#             raise Exception(str(self._error))

#         return self._result
#     def process(self, s : str, timeout : int = 10) :
#         result = self._process_with_timeout(s, timeout)
#         return result

In [4]:
## 본격적으로 데이터 전처리 시작, 데이터 불러오기
original_df = pandas.read_csv("data/ratings_train.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
stopword_list = ['JKC', 'JKG', 'JKB', 'JKV' ,'JKQ', 'JX', 'JC', 'Z_CODA', 'Z_SIOT'] #조사, 사이시옷

df = original_df[['document', 'label']]
kiwi = Kiwi() #객체 생성

for i in tqdm(range(len(df))) :
    text = str(df.iloc[i,0]) #문장 하나씩 가져와서
    text = custom.text_preprocess_kor(text, end_mark=True, chosung=False) #문장의 특문 제거, ?!는 놔두고, 초성은 제거
    if text == " " : #예외 : 전처리 했더니 공백이 된 문자
        df.iloc[i,0] = text 
        continue
    try :
        words = [] #단어 저장 리스트 따로 선언
        temp = kiwi.tokenize(text, z_coda=True, split_complex=True, compatible_jamo=True) #단어 형태소 분석결과를 temp에 저장
        # z-coda : 어미 끝소리 자음 분리
        # split_complex : 합성어, 파생어 추가 분리
        # compatiable_jamo : 따로 떨어진 받침을 처리
        for j in range(len(temp)) :
            if temp[j].tag in stopword_list : #불용어처리
                continue
            words.append(temp[j].form) #단어 부분만 선택해서 words에 저장
        df.iloc[i,0] = " ".join(words) #list를 하나의 문장으로 묶어서 저장
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}") #어떤 문장에서 오류가 생겼는지 확인
        
display(df)

100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [06:38<00:00, 376.87it/s]


,document,label
0,아 더빙 . 진짜 짜증 나 네 목소리,0
1,흠 . 포스터 초딩 영화 줄 . 오버 연기 가볍 지 않 구나,1
2,너무재밓 었 다 그래서 보 는 것 을 추천 하 ㄴ다,0
3,교도소 이야기 이 구먼 . 솔직 히 재미 없 다 . 평점 조정,0
4,사이몬페그 익살 스럽 ㄴ 연기 가 돋보이 었 던 영화 ! 스파이더맨 늙 어 보이 기...,1
...,...,...
149995,인간 이 문제 이 지 . 소 뭔 죄 이 ㄴ가 .,0
149996,평점 이 너무 낮 어서 .,1
149997,이것 이 뭐 ? 한국인 거들먹거리 고 필리핀 혼혈 착하 다 ?,0
149998,청춘 영화 최 고봉 . 방황 우울 하 었 던 날 들 자화상,1


In [6]:
word_len = []
for i in tqdm(range(len(df))) :
    len_ = len(df.iloc[i,0].split())
    word_len.append(len_)
df["word_len"] = word_len

df = df[df["word_len"] > 0]

display(df)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:08<00:00, 17118.92it/s]


,document,label,word_len
0,아 더빙 . 진짜 짜증 나 네 목소리,0,8
1,흠 . 포스터 초딩 영화 줄 . 오버 연기 가볍 지 않 구나,1,13
2,너무재밓 었 다 그래서 보 는 것 을 추천 하 ㄴ다,0,11
3,교도소 이야기 이 구먼 . 솔직 히 재미 없 다 . 평점 조정,0,13
4,사이몬페그 익살 스럽 ㄴ 연기 가 돋보이 었 던 영화 ! 스파이더맨 늙 어 보이 기...,1,28
...,...,...,...
149995,인간 이 문제 이 지 . 소 뭔 죄 이 ㄴ가 .,0,12
149996,평점 이 너무 낮 어서 .,1,6
149997,이것 이 뭐 ? 한국인 거들먹거리 고 필리핀 혼혈 착하 다 ?,0,12
149998,청춘 영화 최 고봉 . 방황 우울 하 었 던 날 들 자화상,1,13


In [7]:
df.to_csv("data/korean_movie_train.txt", sep = "\t", encoding="UTF8")